In [ ]:
from configparser import ConfigParser

config = ConfigParser()
config.read("../config.ini")

root_path = config["admin"]["root_dir"]
bias_mitigation_path = config["admin"]["bias_mitigation"]
import sys

sys.path.insert(1, root_path + bias_mitigation_path + "classes")
from DecisionTree_insession import in_session_decision_tree
from KNN_insession import in_session_KNN
from MLP_insession import in_session_MLP
from MetricsClass import Evaluation
from Plotting import Plots

import pandas as pd
import seaborn as sns
import pickle

Aggregation Bias Mitigation

Group 1: Girls

In [ ]:
# define feature cols
feature_cols = [
    "Erstloesung",
    "Schussel",
    "Erfolg",
    "Schwierigkeit",
    "ist_Schulzeit",
    "MehrfachFalsch",
    "vorher_abgebrochen",
    "Fehler",
    "Klassenstufe",
    "Jahredabei",
    "Testposition__pruefung",
    "Testposition__training",
    "Testposition__version",
    "Art__GK",
    "Art__GR",
    "Art__GZ",
    "Art__K",
    "Art__LB",
    "UserAttribut",
    "OrderNumber",
    "steps",
]

Decision Tree Classifier

In [ ]:
DTE = in_session_decision_tree()
DTE.set_range(2, 61)

# set sex as column names
DTE.set_majority_group("Sex__w")
DTE.set_demographic_category("gender")

DTE.load_matrices_aggregation_bias(
    "05_aggregation_bias_mitigation/gender/matrix_girls", ".pkl", 1
)

DTE.set_feature_cols(feature_cols)
DTE.set_majority_group("girls")
DTE.set_minority_group(None)
metrics_DTE_girls = DTE.loop_matrices("gender/matrix_girls", 5, 1, 2)

KNN

In [ ]:
KNN = in_session_KNN()
KNN.set_range(2, 61)

KNN.set_majority_group("Sex__w")
KNN.set_demographic_category("gender")

KNN.load_matrices_aggregation_bias(
    "05_aggregation_bias_mitigation/gender/matrix_girls", ".pkl", 1
)
KNN.set_feature_cols(feature_cols)
KNN.set_majority_group("girls")
KNN.set_minority_group(None)

metrics_KNN_girls = KNN.loop_matrices("gender/matrix_girls", 2, "uniform")

MLP

In [ ]:
MLP = in_session_MLP()
MLP.set_range(2, 61)

MLP.set_majority_group("Sex__w")
MLP.set_demographic_category("gender")

MLP.load_matrices_aggregation_bias(
    "05_aggregation_bias_mitigation/gender/matrix_girls", ".pkl", 1
)
MLP.set_feature_cols(feature_cols)
MLP.set_majority_group("girls")
MLP.set_minority_group(None)

metrics_MLP_girls = MLP.loop_matrices(
    "gender/matrix_girls", "Adam", "binary_crossentropy", "accuracy", 21, 21
)

Group 2: Boys

Decision Tree Classifier

In [ ]:
DTE = in_session_decision_tree()
DTE.set_range(2, 61)

DTE.set_majority_group("Sex__m")
DTE.set_demographic_category("gender")

DTE.load_matrices_aggregation_bias(
    "05_aggregation_bias_mitigation/gender/matrix_boys", ".pkl", 1
)

DTE.set_feature_cols(feature_cols)
DTE.set_majority_group("boys")
DTE.set_minority_group(None)

metrics_DTE_boys = DTE.loop_matrices("gender/matrix_boys", 5, 1, 2)

KNN

In [ ]:
KNN = in_session_KNN()
KNN.set_range(2, 61)

KNN.set_majority_group("Sex__m")
KNN.set_demographic_category("gender")

KNN.load_matrices_aggregation_bias(
    "05_aggregation_bias_mitigation/gender/matrix_boys", ".pkl", 1
)
KNN.set_feature_cols(feature_cols)
KNN.set_majority_group("boys")
KNN.set_minority_group(None)

metrics_KNN_boys = KNN.loop_matrices("gender/matrix_boys", 2, "uniform")

MLP

In [ ]:
MLP = in_session_MLP()
MLP.set_range(2, 61)

MLP.set_majority_group("Sex__m")
MLP.set_demographic_category("gender")

# parameter False, as not balanced out, as representational bias is not attempted to be mitigated yet
MLP.load_matrices_aggregation_bias(
    "05_aggregation_bias_mitigation/gender/matrix_boys", ".pkl", 1
)
MLP.set_feature_cols(feature_cols)
MLP.set_majority_group("boys")
MLP.set_minority_group(None)

metrics_MLP_boys = MLP.loop_matrices(
    "gender/matrix_boys", "Adam", "binary_crossentropy", "accuracy", 21, 21
)

Model metrics

In [ ]:
## construct dfs for both groups from metric df
metrics = pd.concat(
    [
        metrics_DTE_boys,
        metrics_KNN_boys,
        metrics_MLP_boys,
        metrics_DTE_girls,
        metrics_KNN_girls,
        metrics_MLP_girls,
    ]
)

grouped = metrics.groupby(metrics.subgroup)
df_girls = grouped.get_group("girls")
df_boys = grouped.get_group("boys")
df_boys["model"] = df_boys["model"].replace(["DL"], "MLP")

Evaluation

In [ ]:
# evaluate metrics for fairness
evaluation_obj = Evaluation(metrics)
evaluation_obj.set_demographic_category("gender")
evaluation_obj.set_minority_group("girls")
evaluation_obj.set_majority_group("boys")
df_gender = evaluation_obj.evaluate()

In [ ]:
# show table
table = evaluation_obj.showTable(df_gender)
table

In [ ]:
# save
writer = pd.ExcelWriter("df_gender_AggBias.xlsx", engine="xlsxwriter")
df_gender.to_excel(writer, sheet_name="Gender")
writer.save()